In [2]:
import json
import requests
import time
import pandas as pd

In [1]:
path_dataset = "D:/datasets/flickr-30k-images"
path_captions = "D:/YandexDisk/datasets/flickr-30k-images-metadata.csv"
path_prepared_captions = "D:/flickr-30k-images-metadata.csv"

In [3]:
df = pd.read_csv(path_captions, delimiter='|')
df.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together


## Перевод

Квоты — организационные ограничения, которые можно изменить по запросу в техническую поддержку.
Лимиты — технические ограничения, обусловленные особенностями архитектуры Yandex.Cloud. Изменение лимитов невозможно.

Yandex Translate. Квоты (4 апреля 2021):
Вызовов одного метода API в секунду – 20
Символов, отправленных на перевод, в час – 1 млн

In [134]:
IAM_TOKEN = "_____"
FOLDER_ID = "_____"

BEARER = "Bearer " + IAM_TOKEN
HEADERS = {
    "Content-type": "application/json",
    "Authorization": BEARER
}

def translate_with_yandex(text):
    data = {
        "folder_id": FOLDER_ID,
        "texts": text,
        "targetLanguageCode": "ru"
    }

    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate', headers=HEADERS, data=str(data).encode('utf-8'))
    json_text = json.loads(response.text)

    return json_text["translations"][0]["text"]

In [135]:
translate_with_yandex("cloud")

'облако'

In [21]:
def request_yandex_translate(data, col_name, start_index, end_index):
    start = start_index
    count_iters = 0

    for idx in range(start_index, end_index+1, 20):
        start_time = time.time()

        df_twenty = data[start:idx]
        start = idx

        for idx_twenty in df_twenty.index:
            df_twenty.at[idx_twenty, col_name] = translate_with_yandex(df_twenty.at[idx_twenty, col_name])

        count_iters += 20
        if count_iters > 999:
            data.to_csv("D:/temp.csv", sep='|', encoding='utf-8', index=False)
            count_iters = 0

        time_difference = time.time() - start_time

        if time_difference < 1.0:
            time.sleep(1.01 - time_difference)

## Подсчёт количества знаков

In [1]:
def count_chars_num(data, start_index, end_index, max_count):
    chars_counter = 0
    last_idx = -1

    for idx in range(start_index, end_index+1):
        chars_counter += len(data.iloc[idx, 2])
        if chars_counter > max_count:
            last_idx = idx
            break

    return chars_counter, last_idx

In [23]:
counter, idx = count_chars_num(df, 0, 16460, 1000000)

print(idx)
print(counter)

33234
1000103


## Выбрать по миллиону символов

(1) 0 -- 16460
(2) 16461 -- 33234
(3) 33235 -- 50124
(4) 50125 -- 67337
(5) 67338 -- 80713

In [29]:
first_idx = 16461
second_idx = 33234

df_to_trans = df[first_idx:second_idx]
df_to_trans.shape

(16773, 3)

In [30]:
df_to_trans.to_csv("D:/million-flickr-30k-images-metadata.csv", sep='|', encoding='utf-8', index=False)

## Перевести миллион символов

In [30]:
df_to_trans = pd.read_csv("D:/YandexDisk/datasets/million-flickr-30k-images-metadata.csv.csv", delimiter='|')
df_to_trans.tail(5)

,image_name,comment_number,comment
16768,2547366477.jpg,4,A man and two children at the beach
16769,2547785434.jpg,0,A small and gray dog chases a large bull who i...
16770,2547785434.jpg,1,A brown and white cow jumps away from a black ...
16771,2547785434.jpg,2,A black and white dog is playing with a large ...
16772,2547785434.jpg,3,A small dog chasing after a bucking cow


In [31]:
%%time
request_yandex_translate(df_to_trans, " comment", 0, 16772)

Wall time: 1h 43min 26s


In [49]:
df_to_trans.to_csv("D:/russian-flickr-30k-images-metadata-0.csv", sep='|', encoding='utf-8', index=False)

## Объединить переведённые данные

In [4]:
df1 = pd.read_csv("D:/YandexDisk/datasets/russian-flickr-30k-images-metadata-1.csv", delimiter='|')
df2 = pd.read_csv("D:/YandexDisk/datasets/russian-flickr-30k-images-metadata-2.csv", delimiter='|')
df3 = pd.read_csv("D:/YandexDisk/datasets/russian-flickr-30k-images-metadata-3.csv", delimiter='|')
df4 = pd.read_csv("D:/YandexDisk/datasets/russian-flickr-30k-images-metadata-4.csv", delimiter='|')
df5 = pd.read_csv("D:/YandexDisk/datasets/russian-flickr-30k-images-metadata-5.csv", delimiter='|')

df_concated = pd.concat([df1, df2, df3, df4, df5])

In [5]:
df_concated.shape

(80715, 3)

In [6]:
df_concated.to_csv("D:/russian-flickr-30k-images-metadata.csv", sep='|', encoding='utf-8', index=False)